# Trap definition

In [ ]:
from MOTorNOT.beams import *
from MOTorNOT.coils import *
from MOTorNOT import load_parameters
atom = load_parameters()['atom']

linewidth = 2*np.pi*atom['gamma']

coils = QuadrupoleCoils(radius = 0.08,
                        offset = 0.08,
                        turns = 50,
                        current = 55,
                        axis = 2)
    
mot = SixBeamMOT(power=15e-3, radius=10e-3, detuning=-linewidth/4, handedness=1, field=coils.field)

# mot.plot()


# Solve equations of motion

In [ ]:
from MOTorNOT.integration import *

x0 = -20e-3
v0 = np.linspace(.1, 40, 55)
x, v = generate_initial_conditions(x0, v0, theta=90, phi=60)

sol = Solver(x, v, mot.force, 0.2, dt=1e-4)
atoms = sol.solve()

## find trapped atoms
trapped_atoms = []
for atom in atoms:
    r = np.sqrt(np.square(atom.x.iloc[-1]).sum(axis=0))
    v = np.sqrt(np.square(atom.v.iloc[-1]).sum(axis=0))

    if r < 1e-3 and v < 1e-3:
        trapped_atoms.append(atom)

In [ ]:
## visualize force in phase space
from MOTorNOT.analysis import *
trajectories(mot, trapped_atoms, plane='xy')
phase_space_trajectories(mot, trapped_atoms, axis='x')